In [ ]:
#create folders
import os

folders = [r'data',r'data/embeddings',r'data/features',r'data/truth',r'data/relationships',r'data/prepared_data',r'data/prepared_data/UMAPs']

for folder in folders:
    try:
        os.mkdir(folder)
        print("created " + folder + " folder")
    except:
        print(folder + " folder already exists!")

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import molplotly as molplotly
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler
import plotly.express as px

import glob


#FILES
embedding_files = sorted(glob.glob(r'data/embeddings/*.csv'))
feature_files = glob.glob(r'data/features/*.csv')

#NAMING AND INDEXING LISTS AND DICTIONARIES
model_names = [embedding_files[x][embedding_files[x].rindex('/')+1:embedding_files[x].rindex('.')] for x in range(len(embedding_files))]
model_name_index = dict(zip(model_names,list(range(len(model_names)))))
model_index_name = dict(zip(list(range(len(model_names))),model_names))

#EMBEDDINGS
general_model_embeddings = [pd.read_csv(embedding_files[x]) for x in range(len(embedding_files))]

#global variables for chosen dataset part 1
model_embeddings = general_model_embeddings.copy()
models = dict(zip(model_names,model_embeddings))
all_molecules = model_embeddings[0]['SMILES'].tolist()
molecule_name_index = dict(zip(all_molecules,list(range(len(all_molecules)))))
molecule_index_name = dict(zip(list(range(len(all_molecules))),all_molecules))
molecules = all_molecules.copy()

num_models = len(model_embeddings)
num_molecules = len(all_molecules)

#MOLECULAR PROPERTIES
general_mol_features = pd.read_csv(feature_files[0])

#SIMILARITIES
general_model_similarities = dict(zip(model_names,[pd.DataFrame(cosine_similarity(model_embeddings[x].set_index('SMILES'),(model_embeddings[x].set_index('SMILES')))).rename(columns=dict(zip(range(3282),all_molecules)),index=dict(zip(range(3282),all_molecules))) for x in range(len(model_embeddings))]))

mol_features = general_mol_features.copy()
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_mol_features = mol_features.select_dtypes(include=numerics)
numeric_mol_features = numeric_mol_features.loc[:, (numeric_mol_features != 0).any(axis=0)]
features = list(numeric_mol_features.columns)
features_index = dict(zip(features,list(range(len(features)))))

model_similarities = general_model_similarities.copy()

print("models: " + str(model_names))

Create UMAPs:

In [28]:
import umap

reducer = umap.UMAP()

embedding_data = [model_embeddings[x].drop('SMILES',axis=1).values for x in range(num_models)]
scaled_data = [StandardScaler().fit_transform(embedding_data[x]) for x in range(num_models)]

embeddings = [reducer.fit_transform(scaled_data[x]) for x in range(num_models)]

In [29]:

embedding_smiles = [pd.DataFrame(embeddings[x]).rename(columns={0:'x',1:'y'}).join(model_embeddings[x].reset_index()['SMILES']) for x in range(num_models)]

Test scatter plot of UMAP:

In [ ]:
px.scatter(embedding_smiles[2],x='x',y='y')

Save UMAPs to folder \data\prepared_data

In [31]:
for i in range(num_models):
    embedding_smiles[i].to_csv(r'data/prepared_data/UMAPs/umap_'+model_names[i]+'.csv')

Generate pairwise Spearman's rank correlation coefficient dataframe: (may take a few minutes or fourteen minutes)

In [ ]:
from scipy.stats import spearmanr

spearman_model_avgs = pd.DataFrame(index=range(num_models),columns=range(num_models))
spearman_model_meds = pd.DataFrame(index=range(num_models),columns=range(num_models))
pairwise_spearmanr = pd.DataFrame(columns=(['model1','model2']+list(range(num_molecules))))

for i in range(num_models):
    pairwise_spearmanr.loc[len(pairwise_spearmanr)] = [i,i]+[1]*num_molecules
    for k in range(i):
        data1 = model_similarities[model_index_name[i]]
        data2 = model_similarities[model_index_name[k]]
        print(model_index_name[i]+', '+model_index_name[k])

        spearman_statistics = pd.DataFrame(index=range(2),columns=range(num_molecules))
        for column in range(num_molecules):
            spearman_statistics.iloc[:,column] = spearmanr(data1.iloc[:,column],data2.iloc[:,column])
        pairwise_spearmanr.loc[len(pairwise_spearmanr)] = [i,k]+spearman_statistics.iloc[0,:].tolist()
        pairwise_spearmanr.loc[len(pairwise_spearmanr)] = [k,i]+spearman_statistics.iloc[0,:].tolist()

Save pairwise spearman to folder \data\prepared_data

In [33]:
pairwise_spearmanr.to_csv(r'data/prepared_data/pairwise_spearmanr.csv')

Generate nearest neighbor files: (may also take a few minutes, maybe a long time, like an hour)

In [ ]:
from statistics import mean
from random import sample 
from sklearn.neighbors import NearestNeighbors

data = [model_embeddings[x].reset_index().iloc[:,1:] for x in range(num_models)]

model_count = num_models

overlap_func = pd.DataFrame(columns=['num_nbrs','model1','model2','overlap_percent'])

for i in range(20):
    nbrs = int(0.0004510926*len(all_molecules)*1.5**i)
    if nbrs > 1:
        model_nbrs = [NearestNeighbors(n_neighbors=nbrs, algorithm='ball_tree').fit(data[x].set_index('SMILES')).kneighbors(data[x].set_index('SMILES')) for x in range(model_count)]
        for i in range(model_count):
            for k in range(i):
                overlap_func.loc[len(overlap_func.index)] = [nbrs, i, k, mean([float(len(set(model_nbrs[i][1][mol][1:]).intersection(set(model_nbrs[k][1][mol][1:]))))/float(nbrs) for mol in range(len(data))])]
                print("model " + str(i) + " v " + str(k) + ", # neighbors: " + str(nbrs) + ", avg overlap: " + str(overlap_func.loc[len(overlap_func.index)-1,'overlap_percent']))


Save nearest neighbor overlap melted to folder \data\prepared_data

In [35]:
overlap_func.to_csv(r'data/prepared_data/overlap_func_melted.csv')

In [36]:
overlap_func_melted = pd.read_csv(r'data/prepared_data/overlap_func_melted.csv').iloc[:,1:]
overlap_func_melted['model1']=[model_index_name[overlap_func_melted.loc[x,'model1']] for x in range(len(overlap_func_melted))]
overlap_func_melted['model2']=[model_index_name[overlap_func_melted.loc[x,'model2']] for x in range(len(overlap_func_melted))]
overlap_func_melted['model1'] = overlap_func_melted['model1'] + ', ' + overlap_func_melted['model2']
overlap_func_melted = overlap_func_melted.drop('model2',axis=1).rename(columns={'model1':'models'})
columns = overlap_func_melted['models'].unique()
overlap_func_melted = overlap_func_melted.pivot(index='num_nbrs',columns=['models']).droplevel(0,axis=1)
overlap_func_melted.index = overlap_func_melted.index.astype(int)
overlap_func_melted = overlap_func_melted.loc[:,columns]
for name in model_names:
    overlap_func_melted.insert(len(overlap_func_melted.iloc[0,:]), str(name + ', ' + name), [1]*len(overlap_func_melted))

Save nearest neighbor overlap unmelted to folder \data\prepared_data

In [37]:
overlap_func_melted.to_csv(r'data/prepared_data/overlap_func.csv')

In [ ]:
#number of nearest neighbors to show on UMAP when molecule is clicked, change if desired
num_nbrs = 10

neighbors = pd.DataFrame(index=range(3282))
for i in range(num_models):
    nbrs = pd.DataFrame(NearestNeighbors(n_neighbors=num_nbrs, algorithm='ball_tree').fit(model_embeddings[i].iloc[:,:-1]).kneighbors(model_embeddings[i].iloc[:,:-1])[1])
    nbrs = nbrs.rename(columns={key: key+10*i for key in list(range(10))})
    neighbors = neighbors.join(nbrs)


neighbors

Save nearest neighbors to folder \data\prepared_data

In [39]:
neighbors.to_csv(r'data/prepared_data/nearest_neighbors.csv')

Filter the molecular features here for the next two files:

In [ ]:
#include only numerical features in correlation calculation
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_mol_features = mol_features.select_dtypes(include=numerics)

#       The items below filters out columns to decrease the number of features, use if you have more than 400 features

#delete any columns (features) which only include 0s
numeric_mol_features = numeric_mol_features.loc[:, (numeric_mol_features != 0).any(axis=0)]

#drop all columns with more than the threshhold zeros
zero_threshhold = 3000
test = numeric_mol_features.copy()
drop = []
zeroses = []

for col in test.columns:
    if 0 in list(test.loc[:,col]):
        zeros = test.loc[:,col].value_counts()[0]
        zeroses.append(zeros)
        if zeros > zero_threshhold:
            drop.append(col)

#view the number of zeros the features
px.histogram(zeroses,nbins=100)

In [ ]:
#run if you wish to drop all columns with more than the threshhold of zeros
test = test.drop(drop,axis=1)
features = list(test.columns)

print("number of features used: " + str(len(features)))

Generate correlations between distances between two molecules and differences in their features:

In [42]:
#the higher the sample of molecules you use, the more accurate the calculated correlations, but the longer it will take
sample_size = 2000
output_dict = dict(sample(list(molecule_index_name.copy().items()), sample_size))

model = pd.DataFrame()
model = model_similarities[model_names[0]].copy().loc[list(output_dict.values()),list(output_dict.values())]
model.columns = list(output_dict.keys())
model = model.melt()
model.insert(1,'variable2',model.index % sample_size)
model['variable2'] = list(map(lambda x: list(output_dict.keys())[x],model['variable2']))

model = model.rename(columns={"value":model_names[0]})
for i in range(1,num_models):
    model.insert(len(model.iloc[0,:]),model_names[i],model_similarities[model_names[i]].copy().loc[list(output_dict.values()),list(output_dict.values())].melt().iloc[:,[1]])

In [43]:
model_w_features = model.copy()
for i in range(len(features)):
    model_w_features['∆' + features[i]] = abs(pd.DataFrame(mol_features.loc[model['variable'],features[i]]).reset_index().loc[:,features[i]] - pd.DataFrame(mol_features.loc[model['variable2'],features[i]]).reset_index().loc[:,features[i]])

In [ ]:
corrs = pd.DataFrame(index=model_names,columns=['∆' + item for item in features])
for i in model_names:
    print(i)
    for k in features:
        corrs.loc[i,'∆' + k] = model_w_features.loc[:,i].corr(model_w_features.loc[:,'∆' + k])

corrs

Save property correlations to folder \data\prepared_data

In [45]:
corrs.to_csv(r'data/prepared_data/property_correlations.csv')

Calculate R^2 scores from ML predictions on molecular features from embeddings:

In [46]:
#features used will be the same as features used to calculate correlations from above

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import time

scores = pd.DataFrame(columns=model_names)
times = pd.DataFrame(columns=model_names)

# sample_size = 200
# molecules = sample((all_molecules.copy()),sample_size)
molecules = all_molecules.copy()

properties = features.copy()

for prop in properties:
    for i in model_names:
        then = time.time()
        X = models[i].set_index('SMILES').loc[molecules,:]
        y = mol_features.set_index('SMILES').loc[molecules,prop]

        train_X,val_X,train_y,val_y = train_test_split(X,y,random_state=1)

        model = linear_model.OrthogonalMatchingPursuit()
        model.fit(train_X, train_y)
        melb_preds = model.predict(val_X)
        scores.loc[prop,i] = float(r2_score(val_y, melb_preds))
        print(prop + ', ' + i + ', ' + 'r2: ' + str(r2_score(val_y, melb_preds)))
        now = time.time()
        times.loc[prop,i] = now - then


scores.transpose()

In [ ]:
sns.heatmap(scores[scores>0].astype(float))

Save property predictions r2 scores to folder \data\prepared_data

In [49]:
scores.to_csv(r'data/prepared_data/property_predictions_r2_scores.csv')

That's all!!